In [1]:
import os
import time
import datetime
import pyspark.sql.functions as sf
from uuid import *
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.functions import when
from pyspark.sql.functions import col
from pyspark.sql.types import *
from pyspark.sql.functions import lit
from pyspark import SparkConf, SparkContext
from uuid import * 
from uuid import UUID
import time_uuid 
from pyspark.sql import Row
from pyspark.sql.functions import udf
from pyspark.sql.functions import monotonically_increasing_id
from pyspark.sql.window import Window as W


In [2]:
spark = SparkSession.builder.config('spark.jars.packages','com.datastax.spark:spark-cassandra-connector_2.12:3.1.0').getOrCreate()

In [33]:
def retrieve_company_data(url,driver,user,password):
    sql = """(SELECT id as job_id, company_id, group_id, campaign_id FROM job) test"""
    company = spark.read.format('jdbc').options(url=url, driver=driver, dbtable=sql, user=user, password=password).load()
    return company 

In [30]:
host = 'localhost'
port = '3306'
db_name = 'storagedb'
user = 'root'
password = '12345678'
url = 'jdbc:mysql://' + host + ':' + port + '/' + db_name
driver = "com.mysql.cj.jdbc.Driver"

In [34]:
company = retrieve_company_data(url,driver,user,password)

In [5]:
company.show()

+------+----------+--------+-----------+
|job_id|company_id|group_id|campaign_id|
+------+----------+--------+-----------+
|     2|         1|      10|          1|
|     3|         1|      10|          1|
|     4|         1|      10|          1|
|     5|         1|      10|          1|
|     6|         1|      10|          1|
|     7|         1|      10|          1|
|     8|         1|      10|          1|
|     9|         1|      10|          1|
|    39|         1|    NULL|          1|
|    40|         1|      10|          1|
|    41|         1|      10|          1|
|    42|         1|      10|          1|
|    43|         1|    NULL|          1|
|    44|         1|      10|          1|
|    45|         1|      10|          1|
|    46|         1|      10|          1|
|    47|         1|      10|          1|
|    48|         1|      10|          1|
|    49|         1|      10|          1|
|    50|         1|      10|          1|
+------+----------+--------+-----------+
only showing top

In [26]:
def get_latest_time_cassandra():
    data = spark.read.format("org.apache.spark.sql.cassandra").options(table = 'tracking',keyspace = 'mykeyspace').load()
    cassandra_latest_time = data.agg({'ts':'max'}).take(1)[0][0]
    return cassandra_latest_time

In [35]:
def get_mysql_latest_time(url,driver,user,password):    
    sql = """(select max(lasted_update_time) from events) data"""
    mysql_time = spark.read.format('jdbc').options(url=url, driver=driver, dbtable=sql, user=user, password=password).load()
    mysql_time = mysql_time.take(1)[0][0]
    if mysql_time is None:
        mysql_latest = '1998-01-01 23:59:59'
    else :
        mysql_latest = mysql_time.strftime('%Y-%m-%d %H:%M:%S')
    return mysql_latest 

In [41]:
cassandra_time = get_latest_time_cassandra()
mysql_time = get_mysql_latest_time(url,driver,user,password)
print(cassandra_time,' ',mysql_time)

2023-01-14 21:56:05   2024-05-03 19:15:26


In [43]:
if cassandra_time > mysql_time : 
    print("main task")
else:
    print("No new data found")

No new data found


In [4]:
from cassandra.cluster import Cluster
from cassandra.cqlengine import columns
from cassandra.cqlengine.models import  Model
from cassandra.cqlengine.management import sync_table
from cassandra.cqlengine import connection
from cassandra.query import dict_factory
from datetime import datetime, timedelta
import time
import cassandra
import random
import uuid
import math
import pandas as pd
pd.set_option("display.max_rows", None, "display.max_columns", None)
import datetime
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import mysql.connector

host = 'localhost'
port = '3306'
database = 'storagedb'
user = 'root'
password = '12345678'
url = 'jdbc:mysql://' + host + ':' + port + '/' + database
driver = "com.mysql.cj.jdbc.Driver"

keyspace = 'mykeyspace'
cassandra_login = 'cassandra'
cassandra_password = 'cassandra'
cluster = Cluster()
session = cluster.connect(keyspace)

In [5]:
cnx = mysql.connector.connect(user=user, password=password,
                                        host=host,
                                    database=database)
query = """select distinct(id) as publisher_id from master_publisher"""
mysql_data = pd.read_sql(query,cnx)

C:\Users\hungm\AppData\Local\Temp\ipykernel_19592\3903311637.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  mysql_data = pd.read_sql(query,cnx)


In [6]:
mysql_data

,publisher_id
0,1
1,2
2,3
3,4
4,5
5,9
6,10
7,11
8,12
9,13


In [ ]:
# def generating_dummy_data(n_records,session,user,password,host,db_name):
#     publisher = get_data_from_publisher(user,password,host,db_name)
#     publisher = publisher['publisher_id'].to_list()
#     jobs_data = get_data_from_job(user,password,host,db_name)
#     job_list = jobs_data['job_id'].to_list()
#     campaign_list = jobs_data['campaign_id'].to_list()
#     company_list = jobs_data['company_id'].to_list()
#     group_list = jobs_data[jobs_data['group_id'].notnull()]['group_id'].astype(int).to_list()
#     i = 0 
#     fake_records = n_records
#     while i <= fake_records:
#         create_time = str(cassandra.util.uuid_from_time(datetime.datetime.now()))
#         bid = random.randint(0,1)
#         interact = ['click','conversion','qualified','unqualified']
#         custom_track = random.choices(interact,weights=(70,10,10,10))[0]
#         job_id = random.choice(job_list)
#         publisher_id = random.choice(publisher)
#         group_id = random.choice(group_list)
#         campaign_id = random.choice(campaign_list)
#         ts = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
#         sql = """ INSERT INTO tracking (create_time,bid,campaign_id,custom_track,group_id,job_id,publisher_id,ts) VALUES ({},{},{},'{}',{},{},{},'{}')""".format(create_time,bid,campaign_id,custom_track,group_id,job_id,publisher_id,ts)
#         print(sql)
#         session.execute(sql)
#         i+=1 
#     return print("Data Generated Successfully")

In [16]:
create_time = cassandra.util.uuid_from_time(datetime.datetime.now())

In [14]:
create_time

UUID('6d089af6-09a4-11ef-bb8a-7c6643f85610')